# Etapa 1: Importação das bibliotecas

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
!pip install bert-for-tf2

In [ ]:
!pip install sentencepiece

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import tensorflow_hub as hub

In [ ]:
from tensorflow.keras import layers
import bert

# Etapa 2: Carregamento da base de dados

In [ ]:
drive.mount('/content/drive')

In [ ]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ProjetoIA/training.1600000.processed.noemoticon.csv',
                   header = None,
                   names = cols,
                   engine = 'python',
                   encoding='latin-1')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.drop(['id', 'date', 'query', 'user'],
          axis=1,
          inplace=True)

# Etapa 3: Limpeza dos textos

In [ ]:
def limpar_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text() #Deixar o texto em um formato adequado para fazer a limpeza
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)

  return tweet

In [ ]:
teste = data['text'][0]
teste

In [ ]:
result = limpar_tweet(teste)
result

In [ ]:
data_limpo = [limpar_tweet(tweet) for tweet in data.text] #Utiliza a função para limpar todos os tweets

In [ ]:
data_limpo[0:4]

In [ ]:
data_labels = data.sentiment.values #Ajustando os valores do sentimento positivo
data_labels[data_labels == 4] = 1

# Etapa 4: Tokenização

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1', trainable=False) #Pegar modelo
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() #Adicionando o arquivo de vocabulario
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() #Deixando transformar em minusculo
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("My cat is happy.")

In [ ]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
encode_sentence("My dog is hungry.")

In [ ]:
data_entradas = [encode_sentence(sentence) for sentence in data_limpo]

In [ ]:
data_entradas[0]